# Ollama LLM - Multimodality

이 예제는 Ollama를 이용해서 local 환경에서 다양한 오픈 소스 모델을 서빙할 수 있는 예제입니다.
사용자들은 ollama 사이트에서 특정 오픈소스 모델을 검색해서 사용할 수 있습니다.

* Ollama Website
  * https://ollama.com/search


## Ollama 다운로드 및 설치

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Ollama API 서버 실행
다른 프로그램(애플리케이션)이 모델을 사용할 수 있도록 백그라운드에서 API 서버를 실행

아래 터미널을 클릭해서 창을 열고 아래 명령어를 입력해주세요.
```
ollama serve
```

위의 명령어를 실행했을때 로그가 에러없이 올라가면 정상적으로 ollama API 서버가 실행 된것입니다.

### Ollama 실행 및 모델 다운로드
#### Ollama command 명령어

In [ ]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


#### 다운로드 받은 모델 리스트 확인

In [ ]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:1B    8648f39daa8f    815 MB    21 minutes ago    


#### 모델 다운로드

In [ ]:
!ollama pull gemma3:4B

In [ ]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:4B    a2af6cc3eb7f    3.3 GB    26 seconds ago    
gemma3:1B    8648f39daa8f    815 MB    28 minutes ago    


## Ollama API를 통한 처리

#### Ollama package 설치

In [ ]:
%pip install --upgrade --quiet llama-index-llms-ollama

In [ ]:
from llama_index.llms.ollama import Ollama

## Multi-Modal Support

Ollama는 멀티모달 모델을 지원하며, Ollama LLM 클래스는 기본적으로 이미지를 지원합니다.

이는 채팅 메시지의 콘텐츠 차단 기능을 활용합니다.

여기서는 llama3.2-vision 모델을 활용하여 이미지 관련 질문에 답변합니다. 아직 이 모델이 없다면 "ollama pull llama3.2-vision"을 실행하세요.

In [ ]:
!ollama pull llama3.2-vision

In [ ]:
!wget "https://pbs.twimg.com/media/GVhGD1PXkAANfPV?format=jpg&name=4096x4096" -O ollama_image.jpg

--2025-07-31 02:48:39--  https://pbs.twimg.com/media/GVhGD1PXkAANfPV?format=jpg&name=4096x4096
Resolving pbs.twimg.com (pbs.twimg.com)... 172.64.150.129, 104.18.37.127
Connecting to pbs.twimg.com (pbs.twimg.com)|172.64.150.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255054 (249K) [image/jpeg]
Saving to: ‘ollama_image.jpg’

ollama_image.jpg    100%[===================>] 249.08K  --.-KB/s    in 0.08s   

2025-07-31 02:48:39 (3.15 MB/s) - ‘ollama_image.jpg’ saved [255054/255054]



In [ ]:
from llama_index.core.llms import ChatMessage, TextBlock, ImageBlock
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="llama3.2-vision",
    request_timeout=600.0,
    # Manually set the context window to limit memory usage
    context_window=1024,
)

messages = [
    ChatMessage(
        role="user",
        blocks=[
            TextBlock(text="What type of animal is this?"),
            ImageBlock(path="ollama_image.jpg"),
        ],
    ),
]

resp = llm.chat(messages)
print(resp)